In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math
from torch.utils import data
from torchvision import transforms
import importlib.util
spec = importlib.util.spec_from_file_location("module.name", "/home/arnab/Desktop/dnn-offloading/Models/bdddataloader.py")
bddloader = importlib.util.module_from_spec(spec)
spec.loader.exec_module(bddloader)


def first_block(img_part,model,model_parameters):
    print("\tBefore Conv: {}".format(img_part.size()))
    conv1 = nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3, bias = False)
    print(f"First Block: {conv1}")
    bn1 = nn.BatchNorm2d(64) 
    conv1,bn1 = assign_model_conv([conv1,bn1],0,model,model_parameters)
    activation1 = nn.ReLU(inplace = True)
    pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding = 1)
    return pool1(activation1(bn1(conv1(img_part))))


class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1):
        super(Block, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.stride = stride

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias = False, stride = stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias = False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size = 1, bias = False, stride = stride)
        self.bn3 = nn.BatchNorm2d(out_channels)
        
        self.activation = nn.ReLU(inplace=True)
        
    def forward(self, x, model, model_parameters):
        print("\tBefore Conv: {}".format(x.size()))
        self.conv1,self.bn1 = assign_model_conv([self.conv1,self.bn1],0,model,model_parameters)
        self.conv2,self.bn2 = assign_model_conv([self.conv2,self.bn2],6,model,model_parameters)
        print(f"Block: {self.conv1}")
        print(f"Block: {self.conv2}")
        
        out1 = self.conv1(x)
        out1 = self.bn1(out1)
        out1 = self.activation(out1)
        
        out1 = self.conv2(out1)
        out1 = self.bn2(out1)
        if self.in_channels != self.out_channels or self.stride > 1:
            self.conv3,self.bn3 = assign_model_conv([self.conv3,self.bn3],12,model,model_parameters)
            print(f"Block: {self.conv3}")
            out2 = self.conv3(x)
            out2 = self.bn3(out2)
            return self.activation(out1+out2)
        else:
            return self.activation(out1+x)
        
def assign_model_conv(conv_layers,current_index,model,model_parameters):
    conv_layers[0].weight.data = model[model_parameters[current_index]]
    conv_layers[1].weight.data = model[model_parameters[current_index+1]]
    conv_layers[1].bias.data = model[model_parameters[current_index+2]]
    conv_layers[1].running_mean.data = model[model_parameters[current_index+3]]
    conv_layers[1].running_var.data = model[model_parameters[current_index+4]]
    conv_layers[1].num_batches_tracked.data = model[model_parameters[current_index+5]]
    return conv_layers[0],conv_layers[1]

def assign_model_fc(fc,model,input_units,prev_input_units):
    fc.weight.data = model['fc.weight'][:,prev_input_units: (prev_input_units + input_units)]
    fc.bias.data = model['fc.bias']
    return fc

def create_model_list(model):
    model_list = []
    for key in model:
        model_list.append(key)        
    return model_list

def create_model_dict(cfgs,model_list):
    model_dict = {}
    temp = []
    current_model_list_inx = 0
    for i,v in enumerate(cfgs):
        if len(v) == 2:
            out_channel = v[0]
            num_model_parameters = v[1]
        elif len(v) == 3:
            out_channel = v[0]
            num_model_parameters = v[1]
            stride = v[2]
            
        for j in range(current_model_list_inx,current_model_list_inx+num_model_parameters):
            temp.append(model_list[j])
        model_dict.update({i:temp})
        temp = []
        current_model_list_inx += num_model_parameters
    return model_dict


in_channel = np.zeros((2),dtype=int) # worker = 2
in_channel[0] = 3
in_channel[1] = 3
prev_input_units = 0
def node(NN,img_part,current_layer,BATCH_SIZE,channel,model,fog_node):
    global prev_input_units,in_channel
    out_channel = np.zeros((2),dtype=int) # worker = 2
    weight = None
    bias = None
    cfgs = [(64,6), (64,12), (64,12), (64,12), (128,18,2), (128,12), (128,12), (128,12), (256,18,2), (256,12), (256,12), (256,12), (256,12), (256,12), (512,18,2), (512,12), (512,12)]
    model_list = create_model_list(model)
    model_dict = create_model_dict(cfgs,model_list)
    
    if NN == "conv":
        if len(cfgs[current_layer]) == 2:
            out_channel[fog_node] = cfgs[current_layer][0]
            in_channel[fog_node] = out_channel[fog_node]
            stride = 1
        elif len(cfgs[current_layer]) == 3:
            out_channel[fog_node] = cfgs[current_layer][0]
            stride = cfgs[current_layer][2]
            
        model_parameters = model_dict[current_layer]
        if current_layer == 0:
            out = first_block(img_part,model,model_parameters)
            return out
        else:
            block = Block(in_channels = in_channel[fog_node], out_channels = out_channel[fog_node], stride = stride)
            out = block(img_part,model,model_parameters)
            return out
    elif NN == "ff":
        num_classes = 12
        img_part = torch.flatten(img_part,1)
        
        print("\tBefore ff: {}".format(img_part.size()))
        fc = nn.Linear(512, num_classes)
        print(f"FC: {fc}")
        fc.weight.data = model['fc.weight']
        fc.bias.data = model['fc.bias']
        return fc(img_part)

def adaptive_partitioning(img,partition_size):
    index = partition_size
    temp = img.detach().numpy()
    temp = torch.from_numpy(temp[:,:,index[0]:index[1],:])
    return temp

def partition(NN,img,k,CA,f):
    # intializing variables
    A = None
    partition_size = []
    batch = img.shape[0]
    channel = img.shape[1]
    W = [0] * k  # partition
    Windex = []  # partitioning indexes
    init = [0] * (k + 1)
    CA[0] = 0
    CA = [float(val) for val in CA]

    r = img.shape[2]
    c = img.shape[3]
    
    if r > c:
        m = r
    else:
        m = c
    
    
    # sum of capabilities of all nodes
    C2 = np.sum(CA[:(k + 1)])
    
    for i in range(1,k+1):
        C1 = np.sum(CA[:i+1])
        Pi = C1 / C2
        if NN == "conv":
            init[i] = math.floor(Pi * (m - (f - 1)))
            partition_size.append((init[i-1],init[i]+(f-1)))
        elif NN == "ff":
            init[i] = math.floor(Pi * m)
            partition_size.append((init[i - 1],init[i]))

    return partition_size

def main():
    # Load model
    model = torch.load("/home/arnab/Desktop/Data/resnet34.pt", map_location=torch.device('cpu'))
    
    # Configuration of network
    cfgs = [(64,6), (64,12), (64,12), (64,12), (128,18,2), (128,12), (128,12), (128,12), (256,18,2), (256,12), (256,12), (256,12), (256,12), (256,12), (512,18,2), (512,12), (512,12)]
    
    # Number of workers
    worker = 2
    
    # Capabilities of nodes
    #CA = np.random.randint(1, 10, size=k+1)
    CA = [0.0, 4.0, 6.0]
    print("Fog CA: {}".format(CA))
    
    # Initialize variables
    kernel_filters = None
    current_layer = None
    partition_list = {}
    classify_list = []
    after_part = None
    input_ = None
    channel = None
    final_out = None
    BATCH_SIZE = 1
    IMAGE_DIM = 224
    
    # load images
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(IMAGE_DIM),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])
    
    loader = data.DataLoader(
        bddloader.BDDDataset('/home/arnab/Desktop/Data', train=True, transform=transform),
        batch_size=BATCH_SIZE,
        shuffle=True)
    
    print("Image Size: torch.Size([BATCH_SIZE, CHANNEL, ROW, COL])\n")
    inx = 0
    for img,level in loader:
        print("=> Image : {}".format(inx+1))
        print(img.size())

        # Convolutional NN
        for i in range(len(cfgs)):
            current_layer = i
            if i == 0:
                input_ = img
                channel = input_.shape[1]
                kernel_filters = 7
            else:
                input_ = final_out
                channel = input_.shape[1]
                kernel_filters = 3

            key = "Conv" + str(i)
            print("=> Convolution Layers: {}".format(i))
            after_part = partition("conv",input_,worker,CA,kernel_filters)
            partition_list.update({key : after_part})

            # processing and marging
            final_out = None
            for j in range(len(after_part)):
                img_part = adaptive_partitioning(input_,after_part[j])
                out = node("conv",img_part,current_layer,BATCH_SIZE,channel,model,j)
                print("\tAfter: {}\n".format(out.size()))
                if j == 0:
                    final_out = out
                else:
                    final_out = torch.cat((final_out,out),2)

            print("\tAfter Marge: " + str(final_out.size()))
            
        
        
        # Adaptive average Pool
        avgpool = nn.AdaptiveAvgPool2d((1, 1))
        out = avgpool(final_out)
        print(out.size())
        
        # Fully Connected NN
        current_layer += 1
        input_ = out
        channel = input_.shape[1]
        key = "ff"
        print("=> Fully Connected Layers: ")
        for j in range(worker):
            img_part = input_
            out = node("ff",img_part,current_layer,BATCH_SIZE,channel,model,j)
            print("\tAfter ff: {}\n".format(out.size()))
            m = nn.ReLU()
            out = m(out).data > 0
            out = out.int()
            classify_list.append(out)
            print("{}\n".format(out))
        
        
        classify_final = None
        for i in range(len(classify_list)-1):
            if i == 0:
                classify_final = np.bitwise_or(classify_list[i].numpy()[:], classify_list[i+1].numpy()[:]) 
            else:
                classify_final = np.bitwise_or(classify_final,classify_list[i+1].numpy()[:])

        print("Final Feature Classification: {}".format(torch.Tensor(classify_final).double()))
        
        inx += 1
        break # one image break

if __name__ == '__main__':
    main()

Fog CA: [0.0, 4.0, 6.0]
Image Size: torch.Size([BATCH_SIZE, CHANNEL, ROW, COL])

=> Image : 1
torch.Size([1, 3, 224, 224])
=> Convolution Layers: 0
	Before Conv: torch.Size([1, 3, 93, 224])
First Block: Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
	After: torch.Size([1, 64, 24, 56])

	Before Conv: torch.Size([1, 3, 137, 224])
First Block: Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
	After: torch.Size([1, 64, 35, 56])

	After Marge: torch.Size([1, 64, 59, 56])
=> Convolution Layers: 1
	Before Conv: torch.Size([1, 64, 24, 56])
Block: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
Block: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
	After: torch.Size([1, 64, 24, 56])

	Before Conv: torch.Size([1, 64, 37, 56])
Block: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
Block: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1

	Before Conv: torch.Size([1, 256, 14, 14])
Block: Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
Block: Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
Block: Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
	After: torch.Size([1, 512, 7, 7])

	Before Conv: torch.Size([1, 256, 20, 14])
Block: Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
Block: Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
Block: Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
	After: torch.Size([1, 512, 10, 7])

	After Marge: torch.Size([1, 512, 17, 7])
=> Convolution Layers: 15
	Before Conv: torch.Size([1, 512, 8, 7])
Block: Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
Block: Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
	After: torch.Size([1, 512, 8, 7])

	Before Conv: torch.Size([1, 512